In [170]:
%pip install scipy==1.10.1
%pip install numpy pandas matplotlib scikit-learn gensim nltk pysrt bs4 contractions

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.8 MB/s eta 0:00:00
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a ve

In [171]:
import os
import random
import string

import pysrt
from bs4 import BeautifulSoup
import contractions
import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

nltk.download("stopwords")
nltk.download("punkt")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Downloading package stopwords to /home/av/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/av/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [278]:
def tokenize_subtitle(subtitle_path):
    # Read the subtitle file
    subs = pysrt.open(subtitle_path, encoding='utf-8')

    # Extract text from subtitle objects and remove HTML tags
    text = ' '.join([sub.text for sub in subs])
    soup = BeautifulSoup(text, 'html.parser')
    for font_tag in soup.find_all('font'):
        font_tag.decompose()
    clean_text = soup.get_text()

    clean_text = contractions.fix(clean_text)

    # Tokenize into sentences
    sentences = sent_tokenize(clean_text)

    # Get English stopwords
    stop_words = set(stopwords.words('english') + list(string.punctuation) + ['...', '``', '\'\''])

    # Tokenize each sentence into words and remove stopwords
    tokenized_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
        # filtered_words = [contractions.fix(word) for word in filtered_words] # Remove contractions
        if filtered_words:
            tokenized_sentences.append(filtered_words)
    
    return tokenized_sentences

# Path to the subtitle file
subtitle_path = "social_nw.srt"

# Tokenize the subtitle file and remove stopwords
tokens = tokenize_subtitle(subtitle_path)
for sentence in tokens:
    print(sentence)


['know', 'people', 'genius', 'iqs', 'living', 'china', 'people', 'kind', 'living', 'united', 'states']
['possibly', 'true']
['would', 'account']
['well', 'first', 'awful', 'lot', 'people', 'live', 'china', 'question']
['distinguish', 'population', 'people', 'got', '1600', 'sats']
['know', 'take', 'sats', 'china']
['talking', 'china', 'anymore', 'talking']
['got', '1600']
['yes']
['could', 'sing', 'capella', 'group', 'sing']
['mean', 'actually', 'got', 'nothing', 'wrong']
['could', 'row', 'crew', 'invent', '25', 'pc']
['could', 'get', 'final', 'club']
['get', 'final', 'club']
['know', 'woman', "'s", 'perspective', 'sometimes', 'singing', 'capella', 'group', 'good', 'thing']
['serious']
['hand', 'like', 'guys', 'row', 'crew']
['well']
['kidding']
['yes', 'got', 'nothing', 'wrong', 'test']
['ever', 'tried']
['trying', 'right']
['row', 'crew']
['get', 'final', 'club']
['row', 'crew']
['like', 'whatever', 'delusional']
['maybe', 'sometimes', 'say', 'two', 'things']
['sure', 'one', 'supposed

In [279]:
from gensim.models import Word2Vec
# Train Word2Vec model
model = Word2Vec(tokens,min_count=1)


In [280]:
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extract word embeddings
word_vectors = model.wv

# Get word and corresponding vector
word_vector_list = np.array([word_vectors[word] for word in word_vectors.key_to_index.keys()])
words_list = list(word_vectors.key_to_index.keys())

# Calculate pairwise similarities between word vectors
similarities = cosine_similarity(word_vector_list)

# Perform Affinity Propagation clustering
affinity_propagation = AffinityPropagation(affinity='precomputed')
clusters = affinity_propagation.fit_predict(similarities)

# Print the clusters
for cluster_id in np.unique(clusters):
    cluster_words = [word for idx, word in enumerate(words_list) if clusters[idx] == cluster_id]
    print(f"Cluster {cluster_id}: {', '.join(cluster_words)}")

Cluster 0: going, people, mr., social, 256, nap, 60, 65, shops, juice, bighorn
Cluster 1: would, 's, one, first, sure, page, away, diluted, hit, putting, enough, ago, reasons, conduct, choice, responsible
Cluster 2: get, think, us, school, ownership, free, fine, important, beat, eight, opposed, 3.9, received
Cluster 3: mark, go, say, talking, yet, knew, stock, major, checking, opened, coincidence, inventing, flags, sued, wexner
Cluster 4: good, end, job, concerns, star, usual, bank, mozilla, p.m., annoying, tuna
Cluster 5: sorry, difference, project, desk, top, brooks
Cluster 6: really, new, left, making, wall, unloading, betting, clothes, speaker, creative, certain
Cluster 7: right, need, party, three, keep, half, together, along, bill, oil, texts, load
Cluster 8: take, sir, someone, minute, bitch, deposition, ads, cryptic, hated, older
Cluster 9: two, friends, money, tomorrow, log, enter, likeability, states, identical, third, paranoid
Cluster 10: wait, university, pictures, presiden

In [252]:
from sklearn.cluster import KMeans
import numpy as np

# Extract word embeddings
word_vectors = model.wv

# Get word and corresponding vector
word_vector_list = [word_vectors[word] for word in word_vectors.key_to_index.keys()]
words_list = list(word_vectors.key_to_index.keys())

# Apply K-means clustering
kmeans = KMeans(n_clusters=6)  # You can choose the number of clusters
kmeans.fit(word_vector_list)
clusters = kmeans.labels_.tolist()

# Associate words with their clusters
clustered_words = {cluster: [] for cluster in set(clusters)}
for i, word in enumerate(words_list):
    clustered_words[clusters[i]].append(word)

# Print words in each cluster
for cluster, words in clustered_words.items():
    print(f"Cluster {cluster}: {words}")


Gamora is in Cluster 0: ['ronan', 'plan', 'people', 'thanos', 'gamora', 'see', 'could', 'life', 'first', 'dead', 'kree', 'think', 'galaxy', 'mean', 'last', 'daughter', 'else', 'wife', 'around', 'leg', 'light', 'already', 'retrieve', 'frickin', 'master', 'mr.', 'meet', 'live', 'planet', 'new', 'sell', 'supposed', 'made', 'child', 'prison', 'figure', 'retrieving', 'inside', 'woman', 'number', 'families', 'finally', 'cell', 'kid', 'outlaw', 'across', 'laws', 'ancient', 'slaved', 'pete', 'empire', 'actually', 'stopping', 'cruel', 'barely', 'folks', 'government', 'kevin', 'biggest', 'moons', 'quadrant', 'giving', 'steal', 'apart', 'bomb', 'alone', 'issues', 'quit', 'happens', 'stones', 'subject', 'charge', 'fair', 'teach', 'purse', 'truly', 'paid', 'bag', 'looks', 'cover', 'beat', 'information', 'however', 'best', 'ravager', 'town', 'opportunity', 'smushed', 'holy', 'clear', 'ninja', 'forgot', 'composed', 'attempted', 'fork', 'legendary', 'checking', 'morag', 'emperor', 'easier', 'consider'

In [238]:
# Some analysis
words_to_check = ["ransom", "marta", "blanc", "murderer", "thrombeys", "benoit", "blackmail", "cabrera", "murder", "detective", "richard" ,"harlan", "linda", "walt", "alan", "lawyer", "will", "inheritance", "joni"]
for cluster, words in clustered_words.items():
    lc_words = [w.lower() for w in words]
    for word in words_to_check:
        if word in lc_words:
            print(f"{word} is in Cluster {cluster}: {words}")


murder is in Cluster 0: ['know', "'s", 'well', 'peter', 'want', 'look', 'wait', 'say', 'ever', 'nothing', 'friends', 'guy', 'really', 'tell', 'told', 'much', 'course', 'picked', 'find', 'seen', 'frickin', 'lot', 'throat', 'fight', 'security', 'universe', 'laugh', 'stuff', 'cool', 'terran', 'milano', 'idiots', 'able', 'made', 'chance', 'meant', 'none', 'great', 'figure', 'vermin', 'lower', 'escaped', 'quarnyx', 'signed', 'denarian', 'power', 'culture', 'locked', 'end', 'slaved', 'day', 'behold', 'says', 'watch', 'creepy', 'fake', 'possibly', 'calling', 'tried', 'share', 'dance', 'gave', 'biggest', 'moons', 'necrocraft', 'swear', 'flesh', 'quite', 'coordinates', '100', 'rip', 'crimes', 'hundreds', 'carina', 'apart', 'chop', 'clean', 'close', 'sounds', 'dad', 'craft', 'hard', 'named', 'fit', 'deliver', 'bro', 'hate', 'present', 'purse', 'cross', 'losers', 'bag', 'everyone', 'dey', 'junker', 'billion', 'children', 'information', 'attend', 'old', 'best', 'speak', 'yet', 'smushed', 'ninja', 

In [240]:
# Some analysis
words_to_check = ["Groot", "Rocket", "Gamora", "Thanos", "Quill", "Star-Lord", "xandar", "yondu"]
for cluster, words in clustered_words.items():
    for word in words_to_check:
        if word.lower() in words:
            print(f"{word} is in Cluster {cluster}: {words}")

Rocket is in Cluster 1: ['take', 'let', 'rocket', 'yes', 'dead', 'keep', 'galaxy', 'head', 'ravagers', 'three', 'last', 'stop', 'nebula', 'happened', 'two', 'everybody', 'already', 'turned', 'killing', 'master', 'mr.', 'saying', 'blue', 'better', 'live', 'try', 'illegal', 'five', 'deck', 'code', 'sorry', 'done', 'show', 'supposed', 'battle', 'within', 'contain', 'gets', 'shut', 'fast', 'return', 'rodent', 'drax', 'number', 'fine', 'also', 'sister', 'words', 'kid', 'pilots', 'grateful', 'doors', 'across', 'today', 'pete', 'calls', 'guess', 'future', 'criminal', 'barely', 'nonsense', 'thousand', 'surrounded', 'quadrant', 'every', 'send', 'starboard', 'blowing', 'brain', 'organic', 'group', 'thesaurus', 'alone', 'butts', 'son', 'normal', 'stones', 'sources', 'cases', 'laughing', 'broker', 'green', 'halfway', 'senseless', 'attractive', 'existence', 'allow', 'betrayed', 'fighting', 'gunavian', 'keeping', 'hassle', 'slather', 'compared', 'gone', 'clear', 'teeth', 'interested', 'point', 'smok

In [254]:
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, min_count=1)
model.wv.most_similar('cat')

[('dog', 0.17018885910511017),
 ('woof', 0.004503022879362106),
 ('say', -0.027750348672270775),
 ('meow', -0.04461711645126343)]